In [8]:
from dcicutils import ff_utils
from core.wfr2 import *
from core.wfr_settings import *

my_env = 'fourfront-webdev'
my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)

# different types of exps use different steps at the last step(3).
recipe = [
    [['in%20situ%20Hi-C'], 'hi-c-processing-pairs'],
    [['in%20situ%20Hi-C', 'dilution%20Hi-C'], 'hi-c-processing-pairs'],
    [['micro-C',          'DNase%20Hi-C'],    'hi-c-processing-pairs-nore'],
    [['capture%20Hi-C',   'PLAC-seq'],        'hi-c-processing-pairs-nonorm'],
    [['CHIA-pet',         'TrAC-loop'],       'hi-c-processing-pairs-nore-nonorm']
]

In [9]:
#Choose the recipe element to run the pipeline on
exp_type, step3 = recipe[0]

#Choose the type of operations you want
add_pc = False    #add processed files to 'other processed files
add_tag = False   #add the completed process tag if done with all steps
add_wfr = True   #start missing wfrs

set_url = 'experiment-set-replicates/4DNES2M5JIGV/'
#print set_url

run_sets = [ff_utils.get_metadata(set_url , key=my_auth)]
counter = 0
completed = 0
completed_acc = []

all_sets = len(run_sets)
#run_sets = [i for i in run_sets if "HiC_Pipeline_0.2.5"  not in i.get('completed_processes', [])]
print(str(all_sets)+' total number of sets',
      str(all_sets-len(run_sets))+ ' sets completed')

for a_set in run_sets: 
    print
    counter += 1
    fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size, lab = find_pairs(a_set, my_env)
    # skip based on these conditions
    if not bwa_ref or not chrsize_ref:
        print counter, a_set['accession'], organism, enzyme, 'skipping set with not chrsize/bwa index'
        continue
    if 'nonorm' not in step3:
        if f_size < 15:
            print counter, a_set['accession'], 'skipping small file size', str(f_size) 
            continue
    if 'nore' not in step3:
        if not enz_ref:
            print counter, a_set['accession'], 'skipping not ready NZ', organism, enzyme
            continue
    print counter, a_set['accession'],enzyme, organism,f_size
    part3 = 'done'
    set_pairs = []        
    # cycle through the experiments, skip the ones without usable files
    for exp in fastqpairs.keys():
        if not fastqpairs.get(exp):
            print(exp, 'does not have any fastq pairs')
            continue
        # Check Part 1 and See if all are okay
        exp_bams = []
        part1 = 'done'
        part2 = 'done'
        
        for pair in fastqpairs[exp]:
            #############
            step1_result = get_wfr_out(pair[0], 'bwa-mem 0.2.5', my_auth)
            
            ##run no matter what##
            if add_wfr:
                    # RUN PART 1
                    inp_f = {'fastq1':pair[0], 'fastq2':pair[1], 'bwa_index':bwa_ref}
                    name_tag = pair[0].split('/')[2]+'_'+pair[1].split('/')[2]
                    run_missing_wfr2(step_settings('bwa-mem', organism, lab), inp_f, name_tag,my_auth, my_env)
                    break
        break
print completed
print completed_acc

('1 total number of sets', '0 sets completed')

1 4DNES2M5JIGV DpnII human 380
{u'status': u'RUNNING', u'startDate': u'2018-06-25 20:08:38.141000+00:00', u'name': u'bwa-mem_4DNFIO9OHYUV_4DNFIM62WI62c998fbab-59db-4478-8eb1-b74356d69284', u'ResponseMetadata': {u'RetryAttempts': 0, u'HTTPStatusCode': 200, u'HTTPHeaders': {u'x-amzn-requestid': u'8d0fc53c-78b3-11e8-af38-919a88fb13ab', u'content-length': u'2256', u'content-type': u'application/x-amz-json-1.0'}, u'RequestId': u'8d0fc53c-78b3-11e8-af38-919a88fb13ab'}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:bwa-mem_4DNFIO9OHYUV_4DNFIM62WI62c998fbab-59db-4478-8eb1-b74356d69284', u'stateMachineArn': u'arn:aws:states:us-east-1:643366669028:stateMachine:tibanna_pony', u'input': u'{"app_name": "bwa-mem", "_tibanna": {"run_type": "bwa-mem", "run_id": "4DNFIO9OHYUV_4DNFIM62WI62", "url": "https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:executi